# Alexander Muratov's Book Recommdner

I have often wondered why Goodreads, Amazon, and Audible just don't really do a good job in figuring out which books to recommend to me. Perhaps books are a little too personal and situational compared to movies and music. You invest much more time, and often can't boil down your review to just a number. How can you put a work of classic literature alongside a fun comic book? 

But I decided that I would give it a shot anyway. I decided to try the collaborative filtering algorithm I learned in Andrew Ng's coursera Machine Learning course on a public book rating dataset, and see what I get. Perhaps it would give me some cool ideas for what I should be reading, or at least help me see the challenges in solving this problem.


## Using data from  Book-Crossing Dataset 
http://www2.informatik.uni-freiburg.de/~cziegler/BX/

# Step 1. Data Cleaning 
Here, I am going to take the original raw data files and pull out only the information I need to build my recommender system.
The code presented here is a significantly streamlined version of the first time I actually tried to explore this data. 

In [1]:
import pandas as pd
import numpy as np

#first CSV file gives ratings for users for books by code number, but no titles
data = pd.read_csv("BookData_A/BX-Book-Ratings.csv", encoding='latin-1', sep=';')


In [2]:
#second CSV file gives book titles. Some of the titles have weird characters, so it must be read with latin-1 encoding
#This was a painful thing to figure out. 
bookdata = pd.read_csv("BookData_A/BX-Books.csv", encoding='latin-1', sep=';',error_bad_lines=False) 


b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
C:\Users\Alejandro\Anaconda3\lib\site-packages\IPython\core\interactiveshe

In [3]:
#Merging the two CSV files to get a catalog with book titles and ratings 
catalog = data.merge(bookdata, on='ISBN', how='left')

#Getting rid of non-reviews - entries with a rating of 0
#also getting rid of book titles that are NULL value.
reduced_catalog = catalog[(catalog['Book-Rating']>0) & (catalog['Book-Title'].notnull())]


#check to make sure catalog looks ok
reduced_catalog.head(5)

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
1,276726,0155061224,5,Rites of Passage,Judith Rae,2001,Heinle,http://images.amazon.com/images/P/0155061224.0...,http://images.amazon.com/images/P/0155061224.0...,http://images.amazon.com/images/P/0155061224.0...
3,276729,052165615X,3,Help!: Level 1,Philip Prowse,1999,Cambridge University Press,http://images.amazon.com/images/P/052165615X.0...,http://images.amazon.com/images/P/052165615X.0...,http://images.amazon.com/images/P/052165615X.0...
4,276729,0521795028,6,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,2001,Cambridge University Press,http://images.amazon.com/images/P/0521795028.0...,http://images.amazon.com/images/P/0521795028.0...,http://images.amazon.com/images/P/0521795028.0...
8,276744,038550120X,7,A Painted House,JOHN GRISHAM,2001,Doubleday,http://images.amazon.com/images/P/038550120X.0...,http://images.amazon.com/images/P/038550120X.0...,http://images.amazon.com/images/P/038550120X.0...
16,276747,0060517794,9,Little Altars Everywhere,Rebecca Wells,2003,HarperTorch,http://images.amazon.com/images/P/0060517794.0...,http://images.amazon.com/images/P/0060517794.0...,http://images.amazon.com/images/P/0060517794.0...


In [4]:
mycounts = reduced_catalog["Book-Title"].value_counts()
mycounts.index[0:5]

Index(['The Lovely Bones: A Novel', 'Wild Animus', 'The Da Vinci Code',
       'The Secret Life of Bees', 'The Nanny Diaries: A Novel'],
      dtype='object')

In [6]:
#looks good. Now lets figure out which books are the ones that are reviewed most often to feed to the recommender system 

#lets start with a catalog of 1000 most reviewed books
num_top_books = 1000

countsorted_reviewed_books =  reduced_catalog["Book-Title"].value_counts()
most_reviewedbooks = countsorted_reviewed_books.index[0:num_top_books]
most_reviewedbooks[0:5]

Index(['The Lovely Bones: A Novel', 'Wild Animus', 'The Da Vinci Code',
       'The Secret Life of Bees', 'The Nanny Diaries: A Novel'],
      dtype='object')

In [8]:
#lets also make sure that the ones at the bottom of the list have enough reviews
most_reviewedbooks[-5:-1]

Index(['Personal Injuries',
       'Dying for Chocolate (Culinary Mysteries (Paperback))', 'Guilty as Sin',
       'Love You Forever'],
      dtype='object')

In [9]:
#not bad. Seems like it will be a robust list
#because I am still not totally used to working with dataframes and my algorithm is designed for simple matrices and arrays
#I am going to start migrating some stuff to arrays now
titles = np.array( most_reviewedbooks)

#if someone wants to review the books themselves, uncomment to save text file
#np.savetxt("top_book_titles.txt",titles, fmt='%s')

In [10]:
#reducing the dataframe to only contain data for top X books

further_reduced_catalogCut = reduced_catalog["Book-Title"].isin(titles)
further_reduced_catalog = reduced_catalog[further_reduced_catalogCut]

In [11]:
#now lets get a set of the most active reviewers 

reviewingusers =  further_reduced_catalog["User-ID"].value_counts()

#i don't want any reviewers with fewer than 10 reviews.
top_users = reviewingusers[reviewingusers.values>10]
book_reviewers = np.array(top_users.index)
book_reviewers.shape

(1266,)

In [12]:
#I'm going to subddivide users into a test set and training set.
#as of now I am not actually doing this step, but if I wanted  to validate the recommender system,
#I would use only 70% of the data t otrain the model, and the other 30% to test. 
use_set = np.random.rand(top_users.shape[0]) > 0.7 
test_set = np.invert(use_set)


In [61]:
#so 1266 users for 1000 books. Seems like a reasonable final dataset.
#time to make final cuts on the catalog
final_further_reduced_catalogCut = further_reduced_catalog["User-ID"].isin(book_reviewers)
final_further_reduced_catalog = further_reduced_catalog[final_further_reduced_catalogCut]
final_further_reduced_catalog.head(5)



,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
1456,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...
1474,277427,0061009059,9,One for the Money (Stephanie Plum Novels (Pape...,Janet Evanovich,1995,HarperTorch,http://images.amazon.com/images/P/0061009059.0...,http://images.amazon.com/images/P/0061009059.0...,http://images.amazon.com/images/P/0061009059.0...
1522,277427,0316776963,8,Me Talk Pretty One Day,David Sedaris,2001,Back Bay Books,http://images.amazon.com/images/P/0316776963.0...,http://images.amazon.com/images/P/0316776963.0...,http://images.amazon.com/images/P/0316776963.0...
1543,277427,0345413903,10,The Murder Book,Jonathan Kellerman,2003,Ballantine Books,http://images.amazon.com/images/P/0345413903.0...,http://images.amazon.com/images/P/0345413903.0...,http://images.amazon.com/images/P/0345413903.0...
1578,277427,0385424736,9,The Rainmaker,John Grisham,1995,Doubleday Books,http://images.amazon.com/images/P/0385424736.0...,http://images.amazon.com/images/P/0385424736.0...,http://images.amazon.com/images/P/0385424736.0...


# Step 2. Setting up collaborative filtering
Here, I build a collaborative filtering recommendation system for the book reviews that is based on an assignment from the Stanford Machine Learning Coursera. In that case, prepared film data from IMDB was used. Interestingly, I tracked down several bugs and flaws in the code developed for the assignment and fixed them in this implementation. Will note when applicable.

In [14]:
#at this point, I am going to translate all the reviews in this catalog into a simple matrix containg reviews per user.
#i am going to use a simple numpy matrix since the remainder of my code is prepared for such a use.
# in the future, I will try to stick only  to Pandas framework
#titles = np.array(most_reviewedbooks["Book-Title"])
Y = np.zeros((len(titles), len(book_reviewers)))
print(Y.shape)
num_books = Y.shape[0]


(1000, 1266)


In [15]:
#now we go through the final catalog line by line and fill in the elements of Y.
#this is embarrasing and I'm sure there's a more efficient way to do this, but it's not too slow computationally for this dataset
count = 0
for row in final_further_reduced_catalog.values:
    title_index = np.where(titles==row[3])[0][0]
    user_index = np.where(book_reviewers==row[0])[0][0]   
    Y[title_index][user_index]=row[2]
    count+=1

Insert some additional user input. I personally went through the list of books and rated some.


In [20]:
#creating an extra column for the Y matrix 
my_ratings = np.zeros(num_books)
#opening text file. This text file has the 1000 most reviewed books. One book per row.
#For each book I rated, I assign a numerical score at the end of the line.
ratings_file = open('sasha_top_books.txt')
rating_lines = ratings_file.readlines()
index_ratings_ar = []
ratings_ar = []

#I will use regular expression to tease out the books that have been rated.
import re

for line in rating_lines:
    #look for all lines that end with a 1-2 digit number, and then a newline character
    rating = re.findall('\d+', line[-3:])
    if (len(rating)>0):
        rating_value = int(rating[0])
        
        #is the rating one or two characters?
        if (rating_value>=10):
            stripped_line = line[0:-3]
        else:
            stripped_line = line[0:-2]
        
        #now lets get rid of the rating and only keep the title
        extra_stripped_line = stripped_line.strip()
        
        #make sure this title matches one in our list. also find the index. 
        rating_index = np.where(titles==extra_stripped_line)
        if len(rating_index[0])>0:
            index_ratings_ar.append(rating_index[0][0])
            ratings_ar.append(rating_value)
            print ('I assigned this book: ', stripped_line,'\n This rating: ',rating_value)

#now assigning the reviews to our column vector
count = 0
while (count<len(index_ratings_ar)):
    my_ratings[index_ratings_ar[count]]=ratings_ar[count]
    count+=1
    
Y = np.column_stack((my_ratings,Y))


I assigned this book:  The Da Vinci Code              
 This rating:  1
I assigned this book:  The Nanny Diaries: A Novel        
 This rating:  1
I assigned this book:  Harry Potter and the Chamber of Secrets (Book 2)              
 This rating:  5
I assigned this book:  Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))           
 This rating:  5
I assigned this book:  Angels &amp; Demons            
 This rating:  1
I assigned this book:  Harry Potter and the Prisoner of Azkaban (Book 3)              
 This rating:  5
I assigned this book:  Harry Potter and the Goblet of Fire (Book 4)               
 This rating:  5
I assigned this book:  Harry Potter and the Order of the Phoenix (Book 5)                
 This rating:  5
I assigned this book:  The Fellowship of the Ring (The Lord of the Rings, Part 1)              
 This rating:  8
I assigned this book:  The Golden Compass (His Dark Materials, Book 1)                      
 This rating:  8
I assigned this book:  Ameri

In [21]:
#the algorithm uses a second matrix R to tell if a given book has been reviewed by a given user
R = Y>0
num_users = Y.shape[1]
num_books = Y.shape[0]

In [22]:
#now setting some parameters for the collaborative filtering
#original Coursera version had 10 features, but I find better fits with 20. 
#lambda is the regularization parameter. 10 seems to work well
#prefactor is an extra weight for the random seed. 
prefactor = 1
my_lambda = 10.0
num_features = 20


In [23]:
#do-penalties - we don't want the recommender to get fooled by books that have a high mean score
#after it's been reviewed by only a single user. This is an element I added to the system to improve upon the quality of 
#recommendations given

do_penalties = True

In [25]:
#normalization function  that also returns mean score of each book and total number of reviews 
#for each book. 
#the algorithm works better when the scores are normalized (distance from mean)
def normalizeY(YM):
    cooper = np.copy(YM)
    themeans = []
    thecounts = []
    for arr in cooper:
        cut = arr>0
        themean = np.mean(arr[cut])
        count = len(arr[cut])
        themeans.append(themean)
        thecounts.append(float(count))
        arr[cut] = arr[cut] - themean
    themeans = np.array(themeans)
    thecounts = np.array(thecounts)
    return [cooper, themeans,thecounts]


In [26]:
#normalize the matrix, and find mean number of reviews per book
#books that have fewer reviews than the mean will have their score weighted 
#so that it des not deviate too far form a "neutral" review 

[normY, themeans,thecounts] = normalizeY(Y)
medcount = np.mean(thecounts)
print('mean count ',medcount)



mean count  28.635


In [27]:
#now we initialize matrices for books and users. Each element contains a score per feature.
#The features themselves are basically free variables that will be solved for by the optimizer
num_users = normY.shape[1]
print(num_books,num_users)
X= np.random.randn(num_books, num_features)*prefactor
Theta = np.random.randn(num_users, num_features)*prefactor

print(X.shape)
print(Theta.shape)

#unroll both matrices into a single array to be used by the optimizer
intial_parameters_unrolled  = np.append(np.array(X).reshape(-1), np.array(Theta).reshape(-1))

1000 1267
(1000, 20)
(1267, 20)


In [29]:
#now we define the cost function and the gradient of the cost function
#fully vectorized implementation
def cost_function_matrix_reshape_J(params, theY, theR, num_users, num_books, num_features, the_lambda):
    J = 0
    #unroll the vector back into two feture matrices
    X = params[0:(num_books*num_features)]
    X = X.reshape(num_books, num_features)
    Theta = params[(num_books*num_features):]
    Theta = Theta.reshape(num_users, num_features)

    #multiply the two feature matrices, and see how close they are to the target Y.
    #cost function is the difference of squares. Only considering books that have actual ratings in Y.
    J_sub1 = np.multiply((np.dot(X,Theta.transpose()) - theY), theR)
    J = np.sum(np.multiply(J_sub1,J_sub1)) / 2.0

    #add in regularization
    J +=  the_lambda*np.sum(np.multiply(X,X))/2.0  + the_lambda*np.sum(np.multiply(Theta,Theta))/2.0 
    print('current cost is ',J)

    return J

In [30]:
#fully vectorized implimentation of cost function gradient
def cost_function_matrix_reshape_grad(params, theY, R, num_users, num_books, num_features, the_lambda):

    #unroll the vector back into two feture matrices
    X = params[0:(num_books*num_features)]
    X = X.reshape(num_books, num_features)
    Theta = params[(num_books*num_features):]
    Theta = Theta.reshape(num_users, num_features)
    
    #similar calculation to computing J
    Grad_sub1 = np.dot(X,Theta.transpose())
    Grad_sub2 = np.multiply(Grad_sub1, R)
    Grad_sub3 = Grad_sub2 - theY
    
    #but now there's an extra step to compute the gradient by using the other feature matrix
    X_grad = np.dot(Grad_sub3, Theta)
    Theta_grad = np.dot(Grad_sub3.transpose(), X )
    
    #regularization step
    X_grad += the_lambda*X
    Theta_grad += the_lambda*Theta
    
    #return the results
    return np.append(X_grad.reshape(-1), Theta_grad.reshape(-1))

In [31]:
from scipy.optimize import fmin_cg

#now we run the optimizer fmin_cg to minimize the cost function.

results = fmin_cg(cost_function_matrix_reshape_J,intial_parameters_unrolled,fprime=cost_function_matrix_reshape_grad,\
                      args=(normY,R,num_users,num_books,num_features,my_lambda), maxiter=100,disp=True,full_output=True)

current cost is  550588.404912
current cost is  2.25542654098e+13
current cost is  15626279898.6
current cost is  6853360.02231
current cost is  232987.476623
current cost is  387899.822322
current cost is  155285.065142
current cost is  140730.374131
current cost is  111997.795399
current cost is  196442.902716
current cost is  86362.9668294
current cost is  111567.186883
current cost is  75320.6881834
current cost is  62623.3386808
current cost is  58646.8718187
current cost is  51382.6619335
current cost is  74726.3907013
current cost is  49302.6904062
current cost is  48002.6400847
current cost is  44975.6530498
current cost is  40574.8573859
current cost is  35238.6630966
current cost is  43361.3390472
current cost is  33066.7916324
current cost is  31903.0816402
current cost is  35057.6414503
current cost is  31626.1427182
current cost is  31147.5961828
current cost is  30634.9681049
current cost is  32344.8046549
current cost is  30336.0162975
current cost is  30122.3061919
curr

# Looking at the results

In [56]:
#unroll the results vector back into optimized feature matrices
returns = results[0]
newX = returns[:(num_books*num_features)]
newX = newX.reshape(num_books, num_features)
print(' my newX', newX.shape)
newTheta = returns[(num_books*num_features):]
newTheta = newTheta.reshape(num_users, num_features)
print( 'my newTheta ',newTheta.shape)
p = np.dot(newX,  newTheta.transpose())

 my newX (1000, 20)
my newTheta  (1267, 20)


In [57]:
#add the means back in
my_predictions = p[:,0] + themeans
#this is where i penalize scores of books that have fewer than the mean number of reviews
cut = (thecounts<medcount)

# i set it up such that if the book has no reviews, it is assigned a score of 5
# if the book has more than the mean number of reviews, it is assigned whatever actual scores it got.
# if a book has in between zero and n_mean reviews, it is given a linear combination of  5 and the predicted
# score based on the distance to each
if (do_penalties):
    my_predictions[cut] = ((medcount-thecounts[cut])/medcount) * 5.0 + (thecounts[cut]/medcount)*my_predictions[cut]

In [58]:
#print top 20 best rated book.
fingers = np.argsort(-1*my_predictions)
print("Top 20 books to read")

for i in range(0, 20):
    print ('Book to read: ',titles[fingers][i], '\n Predicted score: ', my_predictions[fingers][i])

Top 20 books to read
Book to read:  The Two Towers (The Lord of the Rings, Part 2) 
 Predicted score:  9.28424152774
Book to read:  A Prayer for Owen Meany 
 Predicted score:  9.27240493228
Book to read:  The Murder Book 
 Predicted score:  9.17792545246
Book to read:  The Amber Spyglass (His Dark Materials, Book 3) 
 Predicted score:  9.07783428088
Book to read:  Anne of Green Gables (Anne of Green Gables Novels (Paperback)) 
 Predicted score:  9.01570086438
Book to read:  The Secret Garden 
 Predicted score:  8.9782634167
Book to read:  Me Talk Pretty One Day 
 Predicted score:  8.97001382114
Book to read:  Griffin &amp; Sabine: An Extraordinary Correspondence 
 Predicted score:  8.96471021668
Book to read:  Slaughterhouse Five or the Children's Crusade: A Duty Dance With Death 
 Predicted score:  8.94734482835
Book to read:  The Green Mile 
 Predicted score:  8.9445561559
Book to read:  The Stand: Complete and Uncut 
 Predicted score:  8.89490935159
Book to read:  The Little Prince 

In [60]:
fingers = np.argsort(my_predictions)
print("As a bonus, here are some books you shouldn't bother reading")
for i in range(0, 20):
    print ('Book not to read: ',titles[fingers][i], '\n Predicted score: ', my_predictions[fingers][i])

As a bonus, here are some books you shouldn't bother reading
Book not to read:  The Da Vinci Code 
 Predicted score:  2.71620665311
Book not to read:  The Nanny Diaries: A Novel 
 Predicted score:  2.89658501358
Book not to read:  Angels &amp; Demons 
 Predicted score:  3.50031604216
Book not to read:  Four Blondes 
 Predicted score:  4.44473565208
Book not to read:  The Bourne Identity 
 Predicted score:  4.52439115761
Book not to read:  Wild Animus 
 Predicted score:  4.9967538973
Book not to read:  Divine Secrets of the Ya-Ya Sisterhood: A Novel 
 Predicted score:  5.02056834301
Book not to read:  Deception Point 
 Predicted score:  5.02299172521
Book not to read:  The Shelters of Stone (Earth's Children Series, No 5) 
 Predicted score:  5.09066397905
Book not to read:  The Celestine Prophecy 
 Predicted score:  5.09366851548
Book not to read:  Free 
 Predicted score:  5.10478557147
Book not to read:  The Little Friend 
 Predicted score:  5.12931919405
Book not to read:  Mr. Murder 

Overall, I am pleased that this algorithm works. It (correctly) recommends books that I have already read and rated highly. If I really only wanted new books to read, I would obviously filter those out, but I kept them in as a sanity check. It recommends more Lord of the Rings book after I said I liked the first one. It also recommends the final His Dark Materials book, after I rated the first two pretty highly.

On the other hand, the level of customization is not exactly top-notch.  I have tried it with various inputs, and it tends to always recommend a few specific titles. A Prayer for Owen Meany, Griffin & Sabine, Anne of Green Gables, and The Secret Garden are favorites. I suspect this sort of collaborative filtering is prone to function in this way. Most people only finish reading something that they are already invested in, and would rate highly. I think it's different than the small amount of investment that people would put in for a movie review. For a second pass at this problem, I would probably try association algorithms using decision trees instead of collaborative filtering. 

Nonetheless, there are probably a few solid recommendations in this list. I am actually pretty intrigued by Griffin & Sabine. This was a fun project! 